In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
datos = pd.read_csv("../Data/Bank Marketing DataSet/bank-additional-full.csv", sep=";")
datos.info()

In [ ]:
datos.rename(columns={"emp.var.rate":"emp_var_rate", "cons.price.idx": "cons_price_idx", "cons.conf.idx":"cons_conf_idx", "nr.employed":"nr_employed"}, inplace=True)
datos.info()

In [ ]:
datos["y"].value_counts()

In [ ]:
indices_no = datos[datos.y=="no"].sample(4640).index
datos_no = datos.iloc[indices_no,]
indices_si = datos[datos.y=="yes"].index
datos_si = datos.iloc[indices_si,]

datos_analysis = pd.concat([datos_no, datos_si])
datos_analysis = datos_analysis.sample(len(datos_analysis))
datos_analysis.reset_index(inplace=True)
datos_analysis.info()


In [ ]:
sns.boxplot(data=datos_analysis, x="y", y="emp_var_rate")

In [ ]:
pd.crosstab(datos_analysis.y, datos_analysis.month)

In [ ]:
col_names_cat = [
    "job",
    "marital",
    "education",
    "default",
    "housing",
    "loan",
    "contact",
    "month",
    "day_of_week",
    "poutcome",
    "y"
]

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(datos_analysis[col_names_cat])
a = pd.DataFrame(one_hot_encoder.transform(datos_analysis[col_names_cat]), columns=one_hot_encoder.get_feature_names_out())
b = datos_analysis.select_dtypes(np.number)

data_trainning = pd.concat([b,a], axis=1)

In [ ]:
input_df = data_trainning.iloc[:,0:63]
output_df = data_trainning["y_yes"]

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(input_df)
normalized_input_df = min_max_scaler.transform(input_df)
normalized_input_df = pd.DataFrame(normalized_input_df, columns=input_df.columns)

In [ ]:
modelo = keras.Sequential()
modelo.add(layers.Dense(10, activation="relu", input_shape=[len(normalized_input_df.keys()), ]))
modelo.add(layers.Dense(10, activation="relu"))
modelo.add(layers.Dense(10, activation="sigmoid"))
modelo.add(layers.Dense(1, activation="softmax"))

alg_entrenamiento = tf.keras.optimizers.Adam(learning_rate=1)
modelo.compile(loss="BinaryCrossentropy", metrics=["mae", "mse", "AUC"], optimizer=alg_entrenamiento)

In [ ]:
modelo.summary()

In [ ]:
modelo.predict(normalized_input_df[1:3])

In [ ]:
history = modelo.fit(
    normalized_input_df,
    output_df,
    epochs=100,
    validation_split = 0.2,
    verbose=1
)

In [ ]:
historico = pd.DataFrame(history.history)
historico["Epochs"] = history.epoch

In [ ]:
ax001 = sns.lineplot(data=historico, x="Epochs", y="loss", label = "Función de error")
ax001 = sns.lineplot(data=historico, x="Epochs", y="val_loss", label = "Función de error en Validación", ax = ax001)
sns.move_legend(ax001, "upper left", bbox_to_anchor=(1, 1))